In [ ]:
!pip install openai

In [46]:
from openai import OpenAI
import requests
import json

This is the function we want to call:

In [ ]:

def brave_search(query):
    url = "https://api.search.brave.com/res/v1/web/search"
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "X-Subscription-Token": "xxxxxxxxxxxxxxxx"
    }
    params = {
        "q": query
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Example usage

search_query = "who won la vuelta in 2024"
results = brave_search(search_query)

if results:
    print(json.dumps(results, indent=4))

Here we "talk" with chatgpt:

In [55]:
messages = [
    {"role": "system", "content": "You are an assistant that searchs on the internet in behalf of the user"},
    {"role": "user", "content": "Who won la vuelta in 2024"}]


In [56]:
client = OpenAI(
api_key="sk-asdfasdfwefwfwfwfwfwf"

)

tools = [

  {
    "type": "function",
    "function": {
      "name": "search_in_internet",
      "description": "User wants to search something in internet",
      "parameters": {
        "type": "object",
        "properties": {
          "search_term": {
            "type": "string",
            "description": "The thing the user wants to search"
          }
        },
        "required": [
          "search_term"
        ]
      }
    }
  }

]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(completion.choices[0].message.tool_calls[0])
tool_call = completion.choices[0].message.tool_calls[0]

ChatCompletionMessageToolCall(id='call_11SVjgQeXkSAd0zAvlQGvERo', function=Function(arguments='{"search_term":"La Vuelta a España 2024 winner"}', name='search_in_internet'), type='function')


Do the actual function calling

In [57]:
arguments = json.loads(tool_call.function.arguments)
search_term = arguments.get('search_term')
search_in_internet_result = brave_search(search_term)


send the result back to the llm to interpret the results

In [58]:


function_call_result_message = {
    "role": "tool",
    "content": json.dumps(search_in_internet_result),
    "tool_call_id": tool_call.id
}

messages.append(completion.choices[0].message)
messages.append(function_call_result_message)



completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)


print(completion.choices[0].message.content)

The winner of the 2024 Vuelta a España was Primož Roglič. This marked his fourth victory in this prestigious cycling race.
